# Odd Behavior in Supernode?

Nate, this notebook gives an example of some behavior I've observed that I suspect implies a bug? Not sure though. 

tl;dr: I made a dyadic graph with very strong planted partition structure. `CliqueExpansionModularity()` and `HyperLouvain()` both give the correct answer no problem, but `SuperNodeLouvain()` fails to move any nodes (and errors, but I suspect that's about not moving any nodes in the first step). 

In [23]:
using Optim 

include("jl/omega.jl")
include("jl/HSBM.jl")
include("jl/hypergraph_louvain.jl")
include("jl/inference.jl")
include("jl/warmstart.jl");

In [24]:
# build a data set

n = 200
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

kmax = 3

function ω(p, α)
    k = sum(p)
    length(p) == 1 ? (1.0*n)^(-α[k]) : (1.0*n)^(-α[kmax + k])
end

α0 = [10.0, 0.5, 10.6, 20.0, 1.8, 20.5]

Ω = buildΩ(ω, α0, kmax)

H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((200,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  191, 192, 193, 194, 195, 196, 197, 198, 199, 200]
  E: Dict{Int64,Dict}
  D: Array{Int64}((200,)) [19, 32, 32, 40, 27, 26, 28, 28, 21, 26  …  33, 25, 24, 31, 34, 25, 35, 41, 21, 24]


In [25]:
# this is a dyadic graph (i.e. it has zero edges of size 1 or 3)

for k = 1:3
    p = mean([length(partitionize(Z[e])) == 1 for e in keys(H.E[k])])
    println("k = $k: $(round(100*p, digits = 0)) % of edges are within a single group.")
end

k = 1: NaN % of edges are within a single group.
k = 2: 100.0 % of edges are within a single group.
k = 3: NaN % of edges are within a single group.


In [26]:
Z_dyadic = CliqueExpansionModularity(H)
println("Dyadic Louvain splits the data into $(length(unique(Z_dyadic))) groups.")

Dyadic Louvain splits the data into 2 groups.


In [27]:
α̂, ll = estimateParameters(H, Z, Ω, α0)
Q = modularity(H, Z_dyadic, Ω; α = α̂)
println("The modularity is Q = $(round(Q, digits = 0))")

The modularity is Q = -29884.0


In [28]:
Z_hyper1 = HyperLouvain(H,kmax,Ω;α=α̂)
println("Hypergraph Louvain splits the data into $(length(unique(Z_hyper1))) groups.")

Q = modularity(H, Z_hyper1, Ω; α = α̂)
println("The modularity is Q = $(round(Q, digits = 0))")


Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
Hypergraph Louvain splits the data into 2 groups.
The modularity is Q = -29884.0


In [29]:
Z_hyper2 = SuperNodeLouvain(H,kmax,Ω;α=α̂)
println("SuperNode Hypergraph Louvain splits the data into $(length(unique(Z_hyper2))) groups.")

Q = modularity(H, Z_hyper2, Ω; α = α̂)
println("The modularity is Q = $(round(Q, digits = 0))")

SuperNode Louvain: Phase 1

Louvain Iteration 1
No nodes moved clusters


UndefVarError: UndefVarError: changed not defined